In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
# Import all required libraries
from fastai.vision.all import *
import albumentations
import PIL.Image
import random

sys.path.append('../scripts')
from model import *
from get_fv import *

In [11]:
# Define the paths to the images, csv with train/valid split info and the trained model
images_data_path = "/home/vinayak/AnimeFaceDataset"
csv_path_name = "/home/vinayak/AnimeFaceDataset/train_valid_split.csv"
model_path = "./models/animeFaceRecognitionModel.pth"

In [15]:
# Read the dataframe and get all the validation files
df = pd.read_csv(csv_path_name)

valid_df = df[df.is_valid == 'valid'].reset_index(drop = True).drop(columns = ['is_valid'])
all_validation_images = valid_df.images.tolist()

In [6]:
# Load the model from trained models
model = network()
model.load_state_dict(torch.load(model_path));
model.eval();

In [19]:
# Extract feature vectors from all the validation files and store them
i2fv = {}

for image in progress_bar(all_validation_images, comment = 'Extracting embeddings...'):
    ipth = Path(f'{images_data_path}/{image}')
    fv = get_fv(ipth, model)
    i2fv[image] = fv

In [23]:
# Pickle these feature vectors and store them to disk
with open('../artefacts/valid_fvs.pkl', 'wb') as f:
    pickle.dump(i2fv, f)
    f.close()